In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

from tqdm import tqdm
tqdm.pandas()

In [2]:
template = pd.read_csv('./dataset/v1/log.log_templates.csv')
n_template = len(template)

In [3]:
def getSeq(log,logid, num):
    logid = list(map(int, logid.split(' ')))
    tar = log['EventId'][logid].values[1:]
    src = log['EventId'][logid].shift(1).values[1:].astype('int')
    
    seq = np.zeros((num+1, num+1))
    for i in range(len(src)):
        seq[src[i],tar[i]] += 1
    seq = seq.reshape(-1)

    seq = seq / seq.sum()
    return seq

In [4]:
label = pd.read_csv('./dataset/v1/label.csv')
train = pd.read_csv('./dataset/v1/train.csv')

label['EventSeq'] = label['LogID'].progress_apply(lambda x: getSeq(train, x, n_template))
for i in tqdm(range(n_template+1)):
    for j in range(n_template+1):
        label['Event_Id_'+str(i)+'->'+str(j)] = label['EventSeq'].apply(lambda x: x[i*(n_template+1)+j])
        
label.drop(['LogID', 'EventSeq'], axis=1, inplace=True)
label.to_csv('./dataset/v1/label_pro_1.csv', index=False)

100%|███████████████████████████████████████████| 70/70 [00:21<00:00,  3.19it/s]


In [5]:
submit = pd.read_csv('./dataset/v1/submit.csv')
test = pd.read_csv('./dataset/v1/test.csv')

submit['EventSeq'] = submit['LogID'].progress_apply(lambda x: getSeq(test, x, n_template))
for i in tqdm(range(n_template+1)):
    for j in range(n_template+1):
        submit['Event_Id_'+str(i)+'->'+str(j)] = submit['EventSeq'].apply(lambda x: x[i*(n_template+1)+j])
        
submit.drop(['LogID', 'EventSeq'], axis=1, inplace=True)
submit.to_csv('./dataset/v1/submit_pro_1.csv', index=False)

100%|███████████████████████████████████████████| 70/70 [00:04<00:00, 14.36it/s]
